# Tool Calling in OpenDXA

This tutorial covers how to make resource methods callable by LLMs through the tool calling system in OpenDXA. Tool calling is specifically designed for classes that inherit from `BaseResource` - it's a core feature that enables LLMs to interact with your resources in a structured way.

## Learning Objectives

By the end of this tutorial, you will understand:

1. What tool calling is and how it works with resources
2. How to use the `tool_callable` decorator in your resource classes
3. How to structure tool parameters and schemas
4. Best practices for implementing callable resource methods
5. How to test and debug tool calls

## Prerequisites

- Understanding of OpenDXA's resource system (`05_resources.ipynb`)
- Familiarity with Python class inheritance
- Basic understanding of decorators

## Important Note

The `tool_callable` decorator is **only valid for classes that inherit from `BaseResource`**. It cannot be used on regular Python classes or standalone functions. This is because tool calling is a core feature of the resource system, providing structured interaction between LLMs and resources.

## 1. Understanding Tool Calling

Tool calling in OpenDXA is specifically designed for resources (classes that inherit from `BaseResource`). When a resource method is marked with `@BaseResource.tool_callable`, it:

1. **Generates a Schema**: Automatically creates a JSON schema from the method's type hints
2. **Exposes the Tool**: Makes the method available for LLMs to call, but only if the method name is in `self._exposed_functions`
3. **Handles Validation**: Ensures parameters match the schema
4. **Manages Execution**: Handles the actual method call and response

Let's look at a simple example:

In [ ]:
from opendxa import LLMResultResource
from typing import Dict, Any, Optional
from pprint import pprint

# Create a simple resource
resource = LLMResultResource()

# Initialize the resource
await resource.initialize()

# Get the tool specifications
tools = resource.as_tool_call_specs()
print("Tool Specifications:")
for tool in tools:
    function = tool["function"]
    print(f"Name: {function["name"]}")
    print(f"Description: {function["description"]}")
    pprint(f"Parameters: {function["parameters"]}")

resource.logger.setLevel(resource.logger.DEBUG)

await resource.final_result(
    content="Hello, world!",
    status="success",
    metadata={"source": "test"},
    error=None
)


## 2. The `tool_callable` Decorator

The `tool_callable` decorator is a class method of `BaseResource` that can only be used within resource classes. Here's how to use it correctly:

In [ ]:
from opendxa.common.resource.base_resource import BaseResource

class ExampleResource(BaseResource):  # Must inherit from BaseResource
    def __init__(self):
        super().__init__()
        self._exposed_functions = {"greet"}  # Only expose the greet method
    
    @BaseResource.tool_callable  # This only works because we inherit from BaseResource
    async def greet(self, name: str) -> str:
        """Greet someone by name."""
        return f"Hello, {name}!"
    
    async def not_exposed(self):
        """This method won't be exposed to LLMs."""
        return "I'm hidden!"

# Create and initialize the resource
resource = ExampleResource()
await resource.initialize()

# Get the tool specifications
tools = resource.as_tool_call_specs()
print("Available Tools:")
for tool in tools:
    print(f"- {tool.name}")

# Try to use the tool
result = await resource.greet("OpenDXA")
print(f"\nTool result: {result}")

## 3. Parameter Schemas

The tool calling system automatically generates schemas from your resource method's type hints. Here's how to structure parameters in your resource methods:

In [ ]:
class SchemaExampleResource(BaseResource):  # Must inherit from BaseResource
    def __init__(self):
        super().__init__()
        self._exposed_functions = {"process_data"}
    
    @BaseResource.tool_callable  # Only works in BaseResource subclasses
    async def process_data(
        self,
        text: str,
        count: int = 1,
        options: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """Process text data with optional parameters.
        
        Args:
            text: The text to process
            count: Number of times to process (default: 1)
            options: Additional processing options
        """
        return {
            "processed_text": text * count,
            "options": options or {}
        }

# Create and initialize the resource
resource = SchemaExampleResource()
await resource.initialize()

# Get the tool specifications
tools = resource.as_tool_call_specs()
print("Tool Schema:")
print(tools[0].inputSchema)

## 4. Best Practices

Here are some best practices for implementing tool-callable methods in your resources:

In [ ]:
class BestPracticeResource(BaseResource):  # Must inherit from BaseResource
    def __init__(self):
        super().__init__()
        self._exposed_functions = {"analyze_text"}
    
    @BaseResource.tool_callable  # Only works in BaseResource subclasses
    async def analyze_text(
        self,
        text: str,
        analysis_type: str = "basic",
        include_metadata: bool = False
    ) -> Dict[str, Any]:
        """Analyze text with configurable options.
        
        Args:
            text: The text to analyze
            analysis_type: Type of analysis (basic/detailed)
            include_metadata: Whether to include metadata
            
        Returns:
            Analysis results with optional metadata
        """
        try:
            # Basic analysis
            result = {
                "word_count": len(text.split()),
                "char_count": len(text)
            }
            
            # Add detailed analysis if requested
            if analysis_type == "detailed":
                result["unique_words"] = len(set(text.split()))
                
            # Add metadata if requested
            if include_metadata:
                result["metadata"] = {
                    "timestamp": "2024-04-14T12:00:00Z",
                    "analysis_type": analysis_type
                }
                
            return result
            
        except Exception as e:
            self.error(f"Analysis failed: {e}", exc_info=True)
            raise

# Create and initialize the resource
resource = BestPracticeResource()
await resource.initialize()

# Test the tool
result = await resource.analyze_text(
    "This is a test sentence for analysis.",
    analysis_type="detailed",
    include_metadata=True
)
print("Analysis Result:")
print(result)

## 5. Testing and Debugging

It's important to test your tool-callable resource methods thoroughly. Here's how to test and debug:

In [ ]:
class TestableResource(BaseResource):  # Must inherit from BaseResource
    def __init__(self):
        super().__init__()
        self._exposed_functions = {"calculate"}
    
    @BaseResource.tool_callable  # Only works in BaseResource subclasses
    async def calculate(
        self,
        operation: str,
        a: float,
        b: float
    ) -> float:
        """Perform a mathematical operation.
        
        Args:
            operation: The operation to perform (add/subtract/multiply/divide)
            a: First number
            b: Second number
            
        Returns:
            The result of the operation
        """
        if operation not in ["add", "subtract", "multiply", "divide"]:
            raise ValueError(f"Invalid operation: {operation}")
            
        if operation == "divide" and b == 0:
            raise ValueError("Cannot divide by zero")
            
        operations = {
            "add": lambda x, y: x + y,
            "subtract": lambda x, y: x - y,
            "multiply": lambda x, y: x * y,
            "divide": lambda x, y: x / y
        }
        
        return operations[operation](a, b)

# Create and initialize the resource
resource = TestableResource()
await resource.initialize()

# Test valid operations
print("Testing valid operations:")
for op in ["add", "subtract", "multiply", "divide"]:
    try:
        result = await resource.calculate(op, 10, 2)
        print(f"{op}(10, 2) = {result}")
    except Exception as e:
        print(f"{op} failed: {e}")

# Test error cases
print("\nTesting error cases:")
try:
    await resource.calculate("invalid", 10, 2)
except Exception as e:
    print(f"Invalid operation error: {e}")
    
try:
    await resource.calculate("divide", 10, 0)
except Exception as e:
    print(f"Divide by zero error: {e}")

## Summary

In this tutorial, we covered:

1. What tool calling is and how it works with resources
2. Using the `tool_callable` decorator in `BaseResource` subclasses
3. Structuring parameters and schemas
4. Best practices for implementation
5. Testing and debugging tools

Key takeaways:

- Tool calling is only available for `BaseResource` subclasses
- Use type hints and docstrings for clear schemas
- Follow best practices for robust implementations
- Test thoroughly to ensure reliability

## Next Steps

1. Try implementing your own tool-callable methods in a resource
2. Explore the capabilities system which builds on tool calling
3. Learn about advanced resource usage patterns